In [3]:
%cd /content/drive/MyDrive/Agriculture App/agriculture-predictor-planner

/content/drive/MyDrive/Agriculture App/agriculture-predictor-planner


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [6]:
#Load the data
df = pd.read_csv("data/merged/crop_soil_weather_merged.csv")

In [ ]:
print(df.info(verbose=True))

In [ ]:
# Convert 'Dist Name' to a categorical variable and 'Dito a numeric code.
df['Dist Name'] = df['Dist Name'].astype('category')
df['State Name'] = df['State Name'].astype('category')
